# 1.04 - Witness Feature Extraction
## **Haunted Places Witness Count**

Numbers extacted using [numberscraper](https://github.com/scrapinghub/number-parser)

**"Haunted_Places_Witness_Count" [datetime]**
- Format: int
- Default Value: 0

**NOTES**:

- Note

    

In [1]:
import pandas as pd
import datefinder
import datetime
import time
import re
from number_parser import parse_ordinal
from number_parser import parse_number
from number_parser import parse


# Reading CSV
df = pd.read_csv("../data/processed/haunted_places.tab", sep = "\t")

In [2]:
def extract_eyewitness(text):
    return parse(text)